In [7]:
import pandas as pd
import numpy as np
import random as rn

recipes = pd.read_csv(r"dishes_with_mapped_tags.csv")
recipes.head()

,index_tasty,id_,name,slug,video_url,thumbnail_url,tags,ratings_negative,ratings_positive,score,...,cuisine_cuban,cuisine_puerto_rican,cuisine_soul_food,cuisine_indigenous,cuisine_laotian,cuisine_hawaiian,method_no_bake_desserts,cuisine_lebanese,cuisine_south_african,cuisine_venezuelan
0,0,868,Homemade Cinnamon Rolls,homemade-cinnamon-rolls,https://vid.tasty.co/output/25812/mp4_640x640/...,https://img.buzzfeed.com/video-api-prod/assets...,"7336057,6854262,5993378,5831534,1280501,124779...",475,17340,98.0,...,0,0,0,0,0,0,0,0,0,0
1,1,6124,Whipped Coffee,whipped-coffee,https://vid.tasty.co/output/163616/landscape_4...,https://img.buzzfeed.com/video-api-prod/assets...,"64487,64470,8091916,6718867,64483,64471,64465,...",777,8483,92.0,...,0,0,0,0,0,0,0,0,0,0
2,2,3094,Fluffy Pancakes,fluffy-perfect-pancakes,https://vid.tasty.co/output/65088/landscape_48...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,"64469,8091748,8091747,8091917,5831534,1280510,...",1099,5009,83.0,...,0,0,0,0,0,0,0,0,0,0
3,3,4693,How to Make Cinnamon Rolls,tasty-101-cinnamon-rolls,https://vid.tasty.co/output/121022/landscape_4...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,"64469,5831534,64483,64484,65843,1280506,124778...",207,6649,97.0,...,0,0,0,0,0,0,0,0,0,0
4,4,1655,Healthy Banana Pancakes,healthy-banana-pancakes,https://vid.tasty.co/output/27116/mp4_640x640/...,https://img.buzzfeed.com/thumbnailer-prod-us-e...,"64469,64463,64470,8091916,1247785,1247788,1247...",591,3853,87.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df = recipes.drop(columns=['index', 'id_', 'slug', 'video_url', 'thumbnail_url', 'tags', 'cook_time', 'prep_time', 'total_time', 'ratings_negative', 'ratings_positive', 'score', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber'])
df.head()

KeyError: "['index', 'cook_time', 'prep_time', 'total_time', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber'] not found in axis"

In [9]:
class DataProcessing:
    @staticmethod
    def pickSample(x, quantity):
        trainingSet = x.sample(quantity)
        validatingSet = x.drop(trainingSet.index)
        return trainingSet, validatingSet
    
    @staticmethod
    def normalization(x):
        values = x.select_dtypes(exclude="object")
        columnNames = values.columns.tolist()
  
        for column in columnNames:
            data = x.loc[:, column]
            maximum = max(data)
            minimum = min(data)
            for row in range(0, len(x), 1):
                xprim = (x.at[row, column] - minimum)/(maximum - minimum)
                x.at[row, column] = xprim

In [10]:
trainingSet, validatingSet = DataProcessing.pickSample(df, 40)

NameError: name 'df' is not defined

In [ ]:
print("Recipes chosen by the user:")
display(trainingSet[['name']].head())

Recipes chosen by the user:


,name
3236,Jalebi As Made By Swasti
4019,Buffalo Chickpea Wraps
897,Easy Juicy Chicken Marsala
3625,Curly Fries
1741,Classic Caesar Salad


In [ ]:
trainingSet = trainingSet.select_dtypes(exclude=["object"])

In [ ]:
class SoftSet:
    @staticmethod
    def addSumsColumn(validatingSet, meanVector):
        sums = []
        testingSet = validatingSet.select_dtypes(exclude=["object"])
        for i in testingSet.values:
            j = 0
            sum = 0
            for element in i:
                sum += element*meanVector[j]
                j += 1
            sums.append(sum)
        return sums
    @staticmethod
    def meanVector(trainingSet):
        meanVector = []
        for column in trainingSet:
            columnValues = trainingSet[column].values
            meanVector.append(round(sum(columnValues)/len(columnValues), 2))
        return meanVector
    @staticmethod
    def calculateAccuracy(meanVector, validatingSetMeanVector):
        vector = np.abs(np.subtract(meanVector, validatingSetMeanVector))
        acc = sum(vector)/len(vector)
        acc = 1 - acc
        return acc

In [ ]:
meanVector = SoftSet.meanVector(trainingSet)

In [ ]:
validatingSet['sums'] = SoftSet.addSumsColumn(validatingSet, meanVector)

In [ ]:
validatingSet = validatingSet.sort_values(by="sums", ascending=False)

In [ ]:
recommendations = validatingSet.iloc[:, :-1].head(15)

NameError: name 'validatingSet' is not defined

In [ ]:
validatingSetMeanVector = SoftSet.meanVector(recommendations.select_dtypes(exclude=["object"]))

In [ ]:
print("Recipes chosen by the recommender:")
display(recommendations[['name']].head())

Recipes chosen by the recommender:


,name
992,One-Pan Maple Rosemary Chicken
3491,Homemade Soft Pretzels
235,Time Saving Sheet-Pan Eggs
1107,Southwestern Veggie-Packed Tater Tot Casserole
3741,Bacon-Wrapped Burger Ring


In [ ]:
acc = SoftSet.calculateAccuracy(meanVector, validatingSetMeanVector)
print("The accuracy is", round(acc*100, 2), "%")

The accuracy is 93.81 %
